In [1]:
import os
import json
import requests
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
import pandas as pd

pd.set_option("display.max_columns", None)
pd.set_option("display.max_colwidth", None)

In [3]:
api_key = os.getenv("API-KEY")
headers = {"Authorization":
           f"Bearer {api_key}"}

In [4]:
r = requests.get("https://lambdaschool.instructure.com/api/v1/courses/495/assignments", headers=headers)

In [81]:
r

<Response [200]>

In [82]:
raw = r.json()

with open("m-m.json", "w") as outfile:
    json.dump(raw, outfile)

In [84]:
m_m_df = pd.read_json("m-m.json")

In [88]:
m_m_df

,id,description,due_at,unlock_at,lock_at,points_possible,grading_type,assignment_group_id,grading_standard_id,created_at,updated_at,peer_reviews,automatic_peer_reviews,position,grade_group_students_individually,anonymous_peer_reviews,group_category_id,post_to_sis,moderated_grading,omit_from_final_grade,intra_group_peer_reviews,anonymous_instructor_annotations,anonymous_grading,graders_anonymous_to_graders,grader_count,grader_comments_visible_to_graders,final_grader_id,grader_names_visible_to_final_grader,allowed_attempts,secure_params,course_id,name,submission_types,has_submitted_submissions,due_date_required,max_name_length,in_closed_grading_period,is_quiz_assignment,can_duplicate,original_course_id,original_assignment_id,original_assignment_name,original_quiz_id,workflow_state,muted,html_url,has_overrides,needs_grading_count,sis_assignment_id,integration_id,integration_data,quiz_id,anonymous_submissions,published,unpublishable,only_visible_to_overrides,locked_for_user,submissions_download_url,post_manually,anonymize_students,require_lockdown_browser
0,8088,,2020-10-30 06:59:59+00:00,NaT,2020-11-21 07:59:59+00:00,1,points,2467,NaN,2020-10-22 18:05:48+00:00,2020-10-22 18:05:49+00:00,False,False,2,False,False,NaN,False,False,False,False,False,False,False,0,True,NaN,True,-1,eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJsdGlfY...,495,Sprint 1 - Track Team Reflection,[online_quiz],True,False,255,False,False,False,NaN,NaN,NaN,NaN,published,True,https://lambdaschool.instructure.com/courses/4...,False,0,NaN,NaN,{},6788,False,True,False,False,False,https://lambdaschool.instructure.com/courses/4...,False,False,False
1,8176,,2020-11-06 07:59:00+00:00,NaT,2020-11-21 07:59:59+00:00,1,points,2467,NaN,2020-10-22 18:05:57+00:00,2020-10-22 18:05:57+00:00,False,False,11,False,False,NaN,False,False,False,False,False,False,False,0,True,NaN,True,-1,eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJsdGlfY...,495,Sprint 2 - Team Reflection,[online_quiz],True,False,255,False,False,False,NaN,NaN,NaN,NaN,published,True,https://lambdaschool.instructure.com/courses/4...,False,0,NaN,NaN,{},6876,False,True,False,False,False,https://lambdaschool.instructure.com/courses/4...,False,False,False
2,8020,,2020-11-13 07:59:00+00:00,NaT,2020-11-21 07:59:59+00:00,1,points,2467,NaN,2020-10-22 18:05:43+00:00,2020-10-22 18:05:43+00:00,False,False,12,False,False,NaN,False,False,False,False,False,False,False,0,True,NaN,True,-1,eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJsdGlfY...,495,Sprint 3 - Team Reflection,[online_quiz],True,False,255,False,False,False,NaN,NaN,NaN,NaN,published,True,https://lambdaschool.instructure.com/courses/4...,False,0,NaN,NaN,{},6721,False,True,False,False,False,https://lambdaschool.instructure.com/courses/4...,False,False,False
3,8080,,2020-10-27 06:59:59+00:00,NaT,2020-11-21 07:59:59+00:00,1,points,2467,NaN,2020-10-22 18:05:48+00:00,2020-10-22 18:05:48+00:00,False,False,24,False,False,NaN,False,False,False,False,False,False,False,0,True,NaN,True,-1,eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJsdGlfY...,495,"Sprint 1, Module 1 - Mentor/Mentee Orientation",[online_quiz],True,False,255,False,False,False,NaN,NaN,NaN,NaN,published,True,https://lambdaschool.instructure.com/courses/4...,False,0,NaN,NaN,{},6780,False,True,False,False,False,https://lambdaschool.instructure.com/courses/4...,False,False,False
4,7887,,2020-10-28 06:59:59+00:00,NaT,2020-11-21 07:59:59+00:00,1,points,2467,NaN,2020-10-22 18:05:36+00:00,2020-10-22 18:05:36+00:00,False,False,25,False,False,NaN,False,False,False,False,False,False,False,0,True,NaN,True,-1,eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJsdGlfY...,495,"Sprint 1, Module 2 - Mentor/Mentee Orientation",[online_quiz],True,False,255,False,False,False,NaN,NaN,NaN,NaN,published,True,https://lambdaschool.instructure.com/courses/4...,False,0,NaN,NaN,{},6617,False,True,False,False,False,https://lambdaschool.instructure.com/courses/4...,False,False,False
5,7851,,NaT,NaT,2020-11-21 07:59:59+00:00,1,points,2467,NaN,2020-10-22 18:05:34+00:00,2020-11-02 19:52:

In [92]:
m_m_df[["id","name"]]

,id,name
0,8088,Sprint 1 - Track Team Reflection
1,8176,Sprint 2 - Team Reflection
2,8020,Sprint 3 - Team Reflection
3,8080,"Sprint 1, Module 1 - Mentor/Mentee Orientation"
4,7887,"Sprint 1, Module 2 - Mentor/Mentee Orientation"
5,7851,"Sprint 1, Module 3 - Mentee's Peer Mentorship Reflection"
6,8092,"Sprint 1, Module 4 - Mentee's Peer Mentorship Reflection"
7,7936,"Sprint 1, Module 3 - Mentor's Peer Mentorship Reflection"
8,7952,"Sprint 1, Module 4 - Mentor's Peer Mentorship Reflection"
9,7914,"Sprint 1, Module 3 - Mentor's Peer Mentorship Reflection, Mentee #2"


In [4]:
r = requests.get("https://lambdaschool.instructure.com/api/v1/courses/495/quizzes/6780/submissions", headers=headers)

In [5]:
r

<Response [200]>

In [7]:
raw = r.json()

with open("submissions-6780.json","w") as outfile:
    json.dump(raw["quiz_submissions"], outfile)

In [4]:
sub_df = pd.read_json("submissions-6780.json")
sub_df.head()

,id,quiz_id,quiz_version,user_id,submission_id,score,kept_score,started_at,end_at,finished_at,attempt,workflow_state,fudge_points,quiz_points_possible,extra_attempts,extra_time,manually_unlocked,validation_token,score_before_regrade,has_seen_results,time_spent,attempts_left,overdue_and_needs_submission,excused?,html_url,result_url
0,56612,6780,95,3164,326129,1,1,2020-10-26 15:06:58+00:00,2020-11-21 07:59:59+00:00,2020-10-29 15:29:16+00:00,1,complete,NaN,1,NaN,NaT,NaN,6f14f6e7c507ab9a81a88fe38fd93e4d49f5febcd4ae89b06ac01e8b83b039ab,NaN,0.0,260538,0,False,False,https://lambdaschool.instructure.com/courses/495/quizzes/6780/submissions/56612,https://lambdaschool.instructure.com/courses/495/quizzes/6780/history?quiz_submission_id=56612&version=1
1,56613,6780,95,453,314219,1,1,2020-10-26 15:07:21+00:00,2020-11-21 07:59:59+00:00,2020-10-27 16:03:13+00:00,1,complete,NaN,1,NaN,NaT,NaN,2e462d5e864ec1cde85437eaa882dead7992900247912e7e8e696c2d74be12cf,NaN,0.0,89753,0,False,False,https://lambdaschool.instructure.com/courses/495/quizzes/6780/submissions/56613,https://lambdaschool.instructure.com/courses/495/quizzes/6780/history?quiz_submission_id=56613&version=1
2,56623,6780,95,616,306786,1,1,2020-10-26 15:32:09+00:00,2020-11-21 07:59:59+00:00,2020-10-26 22:52:04+00:00,1,complete,NaN,1,NaN,NaT,NaN,84d0a735b6c714337e9f3b5bb41575d9e86b2173051df302e9008a22cc6fcfb0,NaN,0.0,26395,0,False,False,https://lambdaschool.instructure.com/courses/495/quizzes/6780/submissions/56623,https://lambdaschool.instructure.com/courses/495/quizzes/6780/history?quiz_submission_id=56623&version=1
3,56646,6780,96,1204,329346,1,1,2020-10-26 16:09:47+00:00,2020-11-21 07:59:59+00:00,2020-10-26 16:10:27+00:00,1,complete,NaN,1,NaN,NaT,NaN,e49ebdcef59229bcc242185fefa0e24d270551543532bc062144a2deaa7c4e52,NaN,0.0,40,0,False,False,https://lambdaschool.instructure.com/courses/495/quizzes/6780/submissions/56646,https://lambdaschool.instructure.com/courses/495/quizzes/6780/history?quiz_submission_id=56646&version=1
4,56649,6780,96,259,311293,1,1,2020-10-26 16:17:03+00:00,2020-11-21 07:59:59+00:00,2020-11-21 07:59:59+00:00,1,complete,NaN,1,NaN,NaT,NaN,b7c4199ce920a429807ca2cf4bd057f807b73882182e67659fe7ed2737414bdb,NaN,NaN,2216576,0,False,False,https://lambdaschool.instructure.com/courses/495/quizzes/6780/submissions/56649,https://lambdaschool.instructure.com/courses/495/quizzes/6780/history?quiz_submission_id=56649&version=1


In [5]:
r = requests.get("https://lambdaschool.instructure.com/api/v1/courses/495/quizzes/6780/submissions/56623/events", headers=headers)
r

<Response [200]>

In [6]:
raw = r.json()["quiz_submission_events"]
for i in raw:
    i["user_id"] = 616

In [7]:
with open("events-56623.json", "w") as outfile:
    json.dump(raw, outfile)

In [8]:
events_df_56623 = pd.read_json("events-56623.json")
events_df_56623

,id,event_type,event_data,created_at,user_id
0,635196,submission_created,"{'quiz_version': 95, 'quiz_data': [{'answers': [{'weight': 100, 'text': 'answer_text', 'id': 2455, 'numerical_answer_type': 'exact_answer', 'exact': 1.0, 'margin': 0.0}, {'weight': 100, 'text': 'answer_text', 'id': 8458, 'numerical_answer_type': 'exact_answer', 'exact': 2.0, 'margin': 0.0}, {'weight': 100, 'text': 'answer_text', 'id': 4060, 'numerical_answer_type': 'exact_answer', 'exact': 3.0, 'margin': 0.0}], 'correct_comments': '', 'incorrect_comments': '', 'question_text': '<div id=""t-formula-bar-input-container""> <div dir=""ltr""> <div id=""t-formula-bar-input""> <p class=""cell-input disabled-input"" dir=""ltr"" role=""combobox"" aria-autocomplete=""list"">How did your orientation go today?</p> <p class=""cell-input disabled-input"" dir=""ltr"" role=""combobox"" aria-autocomplete=""list""><span style=""color: #1d1c1d; font-family: Slack-Lato, appleLogo, sans-serif; font-size: 12pt; font-style: normal; font-weight: 400; text-align: left; text-indent: 0px; white-space: normal; background-color: #ffffff; display: inline !important; float: none;"">Answer on a scale of 1-3, where 1 = below expectations, 2 = met expectations, and 3 = exceeded expectations.</span></p> </div> </div> </div>', 'question_type': 'numerical_question', 'question_name': 'Question 1', 'migration_id': 'mastercourse_16855_145_43098bcecca3bcbf6098697482c7c483', 'points_possible': 1.0, 'assessment_question_migration_id': 'mastercourse_16855_145_f47b767d37e06559ff801f2d253307ba', 'question_bank_name': 'Sprint 1, Module 1 - Mentor/Mentee Orientation', 'question_bank_migration_id': 'ia38a3f5ef54ba1c460d106b564b00163', 'is_quiz_question_bank': True, 'prepped_for_import': True, 'position': 1, 'id': 22635, 'assessment_question_id': None, 'published_at': '2020-10-26T14:44:31+00:00', 'name': 'Question 1'}, {'answers': [], 'correct_comments': '', 'incorrect_comments': '', 'question_text': '<div id=""t-formula-bar-input-container""> <div dir=""ltr""> <div id=""t-formula-bar-input""> <p class=""cell-input disabled-input"" dir=""ltr"" role=""combobox"" aria-autocomplete=""list"">What is one key take-away you've gained from today's session?</p> </div> </div> </div>', 'question_name': 'Question 2', 'migration_id': 'mastercourse_16855_145_d6de3cc9d13e43ae6307a8c7c5930775', 'points_possible': 1.0, 'assessment_question_migration_id': 'mastercourse_16855_145_f47b767d37e06559ff801f2d253307ba', 'question_type': 'essay_question', 'question_bank_name': 'Sprint 1, Module 1 - Mentor/Mentee Orientation', 'question_bank_migration_id': 'ia38a3f5ef54ba1c460d106b564b00163', 'is_quiz_question_bank': True, 'prepped_for_import': True, 'position': 2, 'id': 22636, 'assessment_question_id': None, 'published_at': '2020-10-26T14:44:31+00:00', 'name': 'Question 2'}, {'answers': [], 'correct_comments': '', 'incorrect_comments': '', 'question_text': '<div id=""t-formula-bar-input-container""> <div dir=""ltr""> <div id=""t-formula-bar-input""> <p class=""cell-input disabled-input"" dir=""ltr"" role=""combobox"" aria-autocomplete=""list""><span>Do you have any questions or anything we should know about?</span></p> </div> </div> </div>', 'question_name': 'Question 3', 'migration_id': 'mastercourse_16855_145_90c988693d7535dd83db491b8a2daeee', 'points_possible': 1.0, 'assessment_question_migration_id': 'mastercourse_16855_145_f47b767d37e06559ff801f2d253307ba', 'question_type': 'essay_question', 'question_bank_name': 'Sprint 1, Module 1 - Mentor/Mentee Orientation', 'question_bank_migration_id': 'ia38a3f5ef54ba1c460d106b564b00163', 'is_quiz_question_bank': True, 'prepped_for_import': True, 'position': 3, 'id': 22637, 'assessment_question_id': None, 'published_at': '2020-10-26T14:44:31+00:00', 'name': 'Question 3'}]}",2020-10-26 15:32:10+00:00,616
1,635197,session_started,"{'user_agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.75 Safari/537.36'}",2020-10-26 15:32:10+00:0

In [ ]:
def get_submissions(r):
    data_set = []
    raw = r.json()

    for submission in raw:
        data_set.append(submission)

    if "next" in r.links.keys():
        while "next" in r.links.keys():
            r = requests.get(r.links["next"]["url"], headers=headers)
            raw = r.json()
            for submission in raw:
                data_set.append(submission)
        if "last" in r.links.keys() and r.links['current']['url'] == r.links['last']['url']:
            print('Done!')
    else:
        print("Just one page!")
    return data_set

In [15]:
r = requests.get(f"https://lambdaschool.instructure.com/api/v1/courses/{course_id}/quizzes/{quiz_id}/submissions/{submission_id}/events", headers=headers)
def get_answer(df):
    for i in df["event_data"]:
        if isinstance(i, list) and isinstance(i[0], str)==False:
            if "22635" in i[0].values():
                print(i[0]["answer"])
get_answer(events_df_56623)

2.0


In [32]:
r = requests.get("https://lambdaschool.instructure.com/api/v1/courses/495/quizzes/6780/submission_users", headers=headers)
r

<Response [200]>